In [ ]:
!pip3 install --upgrade boto3   

# Mejores prácticas para diseñar prompts efectivos

In [ ]:
import boto3
import json


In [ ]:
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime', 
    region_name='us-east-1'
)

In [ ]:
def api_bedrock(prompt):
  body = {"prompt": f'Human:{prompt}\n\nAssistant:',
          "max_tokens_to_sample":300,
          "temperature":1,
          "top_k":250,
          "top_p":0.999,
          "stop_sequences":["Human:"]
          }
  kwargs = {
    "modelId": "anthropic.claude-v2",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(body)}
  response = bedrock_runtime.invoke_model(**kwargs)
  response_body = json.loads(response.get("body").read())
  completetion = response_body.get("completion")
  print(completetion)

## Ser claro y conciso


In [ ]:
prompt1="Calcula la suma total de la siguiente secuencia de números: 4, 8, 12, 16."
api_bedrock(prompt1)

In [ ]:
prompt2="¿Cuál es la suma de estos números: 4, 8, 12, 16?"
api_bedrock(prompt2)

## Incluya contexto si es necesario


In [ ]:
text = """Today, AWS announced PartyRock, an Amazon Bedrock Playground. PartyRock is a fun and intuitive hands-on, generative AI app-building playground. In just a few steps, you can create a variety of apps to experiment with generative AI. For example, you could build an app to generate dad jokes on a chosen topic, create the perfect personalized playlist, recommend what to serve based on ingredients in your pantry, analyze and optimize your party budget, or create an AI storyteller to guide your next fantasy role-playing campaign. By building and playing with PartyRock apps, you’ll learn the techniques and capabilities needed to take full advantage of generative AI, including experimenting with various foundation models, building intuition with text-based prompting, and chaining prompts together. PartyRock is powered by Amazon Bedrock, a fully managed service that makes foundation models (FMs) from Amazon and leading AI companies available through an API.PartyRock is built on the belief that all builders should have access to a fun and intuitive tool to get started building with generative AI and be able to share the apps they create to inspire others. You don’t need to know how to write a single line of code to start creating your own apps with PartyRock. Access to PartyRock is also straightforward through its web-based UI, which is separate from the AWS Management Console and does not require an AWS account. Using one-step tools, builders can share apps they’ve created using PartyRock with friends or publish links on their favorite social media using. For a limited time, AWS offers new PartyRock users a free trial without the need to provide a credit card. Everyone can build AI apps with PartyRock."""

In [ ]:
prompt1="Summarize this article: "+text
api_bedrock(prompt1)

In [ ]:
prompt2="Provide a summary of this article to be used in a blog post: "+text
api_bedrock(prompt2)

## Utilice directivas para el tipo de respuesta deseado


In [ ]:
prompt1="¿Cuál es la capital de Nueva York? Proporcione la respuesta en una oración completa."
api_bedrock(prompt1)

In [ ]:
prompt2="¿Cuál es la capital de Nueva York?"
api_bedrock(prompt2)

## Considere el resultado en el mensaje

In [ ]:
prompt1="Calcula el área de un círculo con un radio de 3 pulgadas"
api_bedrock(prompt1)

In [ ]:
prompt2="Calcula el área de un círculo con un radio de 3 pulgadas (7,5 cm). Redondea tu respuesta al número entero más cercano."
api_bedrock(prompt2)

## Iniciar mensajes con una pregunta

In [ ]:
evento = """ En septiembre de 1666, un pequeño incendio comenzó en la panadería del panadero Thomas Farriner en Pudding Lane, en Londres. Sin embargo, debido a las condiciones secas y al viento fuerte, el fuego se extendió rápidamente a través de las casas de madera apiñadas en la ciudad. Dentro de pocas horas, el fuego se había extendido desde Pudding Lane hasta la catedral de San Pablo. Al día siguiente, el fuego había arrasado la mayor parte de la ciudad dentro de las murallas. Durante casi una semana, el fuego ardió sin control. Solo se detuvo cuando el viento cambió de dirección y los esfuerzos para demoler edificios crearon calles de fuego para detener su propagación.
Cuando finalmente terminó, el Gran Incendio de Londres había destruido 13,200 casas, 87 iglesias parroquiales, St. Paul's Cathedral y muchos edificios gubernamentales. Se estima que la pérdida de vidas humanas fue de solo unas pocas docenas, pero decenas de miles de londinenses se quedaron sin hogar. Fue uno de los peores desastres en la historia de Inglaterra. El incendio tuvo un impacto duradero en Londres y en Inglaterra. Sirvió para impulsar la reconstrucción de Londres con amplias calles y edificios de piedra en lugar de madera, reduciendo así el riesgo de futuros incendios a gran escala. También llevó a la creación del London Building Acts, uno de los primeros códigos de construcción del mundo."""

In [ ]:
prompt1= "¿Por qué ocurrió el evento en los tags <evento></evento>? Explícalo en tres frases. <evento>"+evento+"</evento>"

api_bedrock(prompt1)

In [ ]:
prompt2= "Resume  el evento en los tags <evento></evento>. <evento>"+evento+"</evento>"

api_bedrock(prompt2)

## Proporcione una respuesta de ejemplo (one shot)

In [ ]:
review_negativo =  """Compré el nuevo teléfono X pero ha sido una decepción total. La batería dura apenas unas horas y la cámara toma fotos borrosas. Además se cuelga constantemente. No puedo recomendar este producto."""

In [ ]:
prompt1= "Determine el sentimiento de esta publicación en las redes sociales:"+review_negativo
api_bedrock(prompt1)

In [ ]:
prompt2 = f"""Determine el sentimiento de la siguiente publicación en las redes sociales utilizando estos ejemplos:
post: “Muy buen producto, me encanta.” sentimiento: Positivo
post: “Odio cuando se agota la batería de mi teléfono.” sentimiento:  Negativo
post: "{review_negativo}" sentimiento:  """
api_bedrock(prompt2)